# Workshop 1
Hi and welcome to the first of a series of workshops on using MODFLOW-6 with Flopy.

## This workshop agenda
1. Introductions and outline (10 minutes)
2. Setup, environment configuration and downloads (10 minutes, wishful)
3. Flopy Github and MF6 Github (10 minutes)
4. Simulations and models (10 minutes)
5. Setting up your first simulation - the mfsim.nam file (10 minutes)
6. Setting up a model - the model.nam file (10 minutes)
7. Temporal discretisation - the tdis package (20 minutes)
8. Spatial discretisation - DIS, DISV and DISU packages (40 minutes)

    a.2D (Dupuit-Forcheimer)

    b.2D -Xsection

    c.3D


# Imports
These are someof the most common libraries that i use when working on building a model with flopy. You will notice that some libraries are imported with an alias. Hopefully you are all familiar with the reasons for this. If not don't worry it will become clearer later. I find it useful to always check which version of the library you are using in case you get any issues with errors that may be specific to the version of the library that you are using.

In [1]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import flopy
import pyemu

print(f"Pandas version = {pd.__version__}")
print(f"Numpy version = {np.__version__}")
print(f"GeoPandas version = {gpd.__version__}")
print(f"Flopy version = {flopy.__version__}")
print(f"Pyemu version = {pyemu.__version__}")

Pandas version = 2.0.3
Numpy version = 1.25.2
GeoPandas version = 0.13.2
Flopy version = 3.4.2
Pyemu version = 1.3.2


# Setting up some folders to dump things in later
We will be creating files and perhaps using files from other utilities so it is nice to have things organised. You can do this however you want but as a simple example you can follow what I do below. Lets first take a look at where we are on the computer by using the os library.

In [2]:
os.getcwd()

'c:\\Working\\NCGRT_IntroFlopy'

The path above is where we are working right now. Lets make a new folder specific to this workshop.

In [3]:
ws1 = os.path.join('workshop_1') # here we are making a path not creating the folder
if os.path.exists(ws1): # here we are asking if the path exists on the computer. 
    shutil.rmtree(ws1)# if it does exist, delete it and all the files in it
    os.mkdir(ws1) # then remake it
else:
    os.mkdir(ws1) # if it doesn't exist then make the folder

Lets make a few extra paths to folders that we will use later. Notice how the workshop_1 folder path (which is ws1) is now the first entry in the join chain. This means that the paths we are making will be sub-directories of the workshop_1 folder.

In [4]:
gis_f = os.path.join(ws1,'GIS') # creating a sub-directory path for our gis input/output
model_f = os.path.join(ws1,'model') # creating a sub-directory path for our model input/output
plots_f = os.path.join(ws1,'plots') # creating a sub-directory path for our plots

In [5]:
print(gis_f)
print(model_f)
print(plots_f)

workshop_1\GIS
workshop_1\model
workshop_1\plots


Now we will make a few sub-directories inside our workshop folder by using a for loop from a list of strings called subdirectory_list. Once you execute the code below check you folder for the subdirectories.

In [6]:
subdirectory_pathlist = [gis_f,model_f,plots_f] # this is our list of sub-directory paths, each one is a path string
for path in subdirectory_pathlist: # for each path in the list above
    if os.path.exists(path): # check if the folder already exists
        pass # if it does exist then do nothing
    else:
        os.mkdir(path) # otherwise make the folder

# Setting up a MF6 simulation
The simulation namefiel is required for all MF6 modelling work. What you are creating next is the mfsim.nam file. This is the file at the highest level when it comes to all modelling work in you specfic project using mf6. The mfsim.nam file is what the MF6 executable will look for as soon as you try to run a model in you project folder.

In [7]:
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f) # this one is mandatory if you want to make sure that the model files are created in the folder we want

In [8]:
sim

sim_name = MySim
sim_path = c:\Working\NCGRT_IntroFlopy\workshop_1\model
exe_name = mf6

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = MySim



Lets see if we can write the mfsim.nam file using Flopy's package write function.

In [9]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


AttributeError: 'NoneType' object has no attribute 'write'

Hmm, looks like it failed trying to find the tdis package object but it did succeed in writing the mfsim.nam file before failing. The write_simualtion() method expects a timing package object to be present or it will fail. We'll discuss this more later but for now let's take a quick peek at the mfsim.nam file. You can execute the code below to view the contents of the file or alternativley open the file in its folder with your favourite text editor. Do you know where to look for the file?

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"mfsim.nam"))]

Open the mf6io.pdf document to see what else could be in this file if we did things a bit differently.
We will discuss what the other sections are in the document and whether you add them explicitly to the sim object of if Flopy will handle that for you.

Things to think about.

Can I remake the object? Can a simulation have more than one simulation object? Are there any conflicts with options? What happens if enter something incorrectly? Try to remake the sim object with all options turned on.

In [10]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f, # this one is mandatory if you want to make sure that the model files are created in the folder we want
                             continue_=True,
                             nocheck=True,
                             lazy_io=True,
                             memory_print_option="ALL",
                             write_headers=False) 

In [11]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


AttributeError: 'NoneType' object has no attribute 'write'

In [12]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"mfsim.nam"))]

BEGIN options
  CONTINUE
  NOCHECK
  MEMORY_PRINT_OPTION  all
END options

BEGIN exchanges
END exchanges



Wait a minute what about maxerrors?

In [13]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f, # this one is mandatory if you want to make sure that the model files are created in the folder we want
                             continue_=False,
                             nocheck=False,
                             lazy_io=False,
                             memory_print_option="ALL",
                             write_headers=False,
                             maxerrors=20) 

TypeError: MFSimulation.__init__() got an unexpected keyword argument 'maxerrors'

Out of options. Time to go to GitHub or dig into the source code of the library installed on your computer. What you will find is that the installed version does not have the maxerrors option available but the online develop branch version does. This is likely to change in the future. This does not mean you can't use the option because it already exists in MF6. It does mean that you can't use it through flopy because it is in development. For the purposes of this workshop we don't need it but this brief diversion is handy to demonstrate that the software you are using is being developed on a continuous basis. This includes both MF6 and Flopy. New features added to MF6 may take a while to become available in Flopy. So you need to know how to edit the input files manually, which is where the mf6io.pdf becomes your primary resource. 

What if I want to change something in the sim object after it has already been created. There are methods that allow you to access settings directly but for these workshops we're going to keep it as simple as possible so just recreate the object with new settings. but be sure to keep the object names the same. Otherwise you may not be replacing the previous object but attempting to add another one.

In [14]:
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f) # this one is mandatory if you want to make sure that the model files are created in the folder we want

Thats basically all you need to know for the sim object in order to create it. But what if you have an existing model and you want to load it instead. The Flopy GitHub introduction model is available as in import for use with the sim pacakge that you just created. Execute the following block of code to import the function and then run it by passing in your sim object.

In [15]:
from helpers import ws1_mod

In [16]:
ws1_mod(sim)

building tdis package
building ims package
building gwf package
building dis package
building ic package
building npf package
building chd package
building oc package


So you've just built a complete model. Take a quick look at the sim object now. Then open up the simulation workspace folder. Note that you haven't written your model files yet. Lets take a look at what the sim object looks like now.

In [17]:
sim

sim_name = MySim
sim_path = c:\Working\NCGRT_IntroFlopy\workshop_1\model
exe_name = mf6

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = MySim


###################
Package MySim.tdis
###################

package_name = MySim.tdis
filename = MySim.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = MySim


###################
Package ims_-1
###################

package_name = ims_-1
filename = MySim.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = MySim


@@@@@@@@@@@@@@@@@@@@
Model MyModel
@@@@@@@@@@@@@@@@@@@@

name = MyModel
model_type = gwf6
version = mf6
model_relative_path = .

###################
Package dis
###################

package_name = dis
filename = MyModel.dis
package_type = dis
model_or_simulation_package = model
model_name = MyModel


###################
Package ic
#######

Now to write your simulation.

In [18]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model MyModel...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package chd_0...
INFORMATION: maxbound in ('gwf6', 'chd', 'dimensions') changed to 2 based on size of stress_period_data
    writing package oc...


Have a look at your model folder now. The model input files should be there - but no output files just yet.
Now lets run the simulation.

In [19]:
sim.run_simulation()

FloPy is using the following executable to run the model: ..\..\..\..\mffiles\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.4.2 06/28/2023

   MODFLOW 6 compiled Jul 05 2023 20:31:54 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Govern

(True, [])

Have a look at the folder again. If your model completed succesfully. You should now see output files in addition to your input files. Great now we can look at how to load an existing model using the simualtion object.

In [20]:
sim = flopy.mf6.MFSimulation.load(sim_ws=model_f) 
# note that for MF6 you generally only have to provide the path to the model
# It will automatically search for the mfsim.nam file in that location 
# then load the model from that.

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package chd...
    loading package oc...
  loading solution package mymodel...


If the model was not created by Fopy but some other GUI then you may have trouble loading all packages.
Again this is where you need the mfio.pdf to help you out because you may be able to load the package with a minor tweak to the input file.
If it requires major modifications the perhaps load without that package using the load only options.

In [21]:
load_list = ["npf"]
sim = flopy.mf6.MFSimulation.load(sim_ws=model_f, load_only=load_list) 
#note that there are certain packages that it will always load irrespective of what you declare as load only.

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    skipping package ic...
    loading package npf...
    skipping package chd...
    skipping package oc...
    skipping package ims6...


Once you load the model and whatever packages you want, then you can "get" the simualtion level package objects from the simulation object. Note only simulation level package objects. Accessing loaded model level package objects requires that you first "get" the loaded model object from the loaded simualtion. We will do more with the model object later but for completeness we will access both simulation and model level package objects here.

In [22]:
tdis = sim.get_package('tdis') # a simualtion level package object
gwf = sim.get_model("MyModel") # get the model object in order to access model level package
npf = gwf.get_package("NPF") # get the loaded npf package object and assign it to a variable

In [23]:
# check that the variable type mtaches the loaded package
type(npf)

flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf

Now we are going to clear all that and start again but this time we will make our own time discretization package.

In [24]:
for x in [sim,gwf,npf,tdis]: # loop through a list of objects we want to remove
    del x # use the del statement to remove the objects, this clears them from memory
flist = [x for x in os.listdir(model_f)] # gets a list of all the files in the model folder
_ = [os.remove(os.path.join(model_f,f)) for f in flist] # removes all the files in the model folder, faster as list comprehension? Maybe.
# now check that it worked
flist = [x for x in os.listdir(model_f)] # get the files again
flist # should see an empty list

[]

The imporatnt thing to note with the creation of the tdis object is that you are passing in the simulation object. Only the simulation level package objects require that you pass in a sim object.

In [25]:
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f) # this one is mandatory if you want to make sure that the model files are created in the folder we want
tdis = flopy.mf6.ModflowTdis(sim) # will give you a basic tdis object set for single steady-state stress period

In [26]:
tdis

package_name = MySim.tdis
filename = MySim.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = MySim

Block dimensions
--------------------
nper
{internal}
(1)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([(1., 1, 1.)],
          dtype=[('perlen', '<f8'), ('nstp', '<i4'), ('tsmult', '<f8')]))



That may be all you need but lets try to make a slightly better one specifying time units and our model start datetime as ISO 8601 format. Note also that it will warn you that it detects an existing timing package

In [27]:
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00")
tdis

package_name = MySim.tdis
filename = MySim.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = MySim

Block options
--------------------
time_units
{internal}
('days')

start_date_time
{internal}
('2023-12-31t00:00:00')


Block dimensions
--------------------
nper
{internal}
(1)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([(1., 1, 1.)],
          dtype=[('perlen', '<f8'), ('nstp', '<i4'), ('tsmult', '<f8')]))



In [28]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


What if we have a transient model? Lest start with a transient single stress period model. Its goign to run for a year and have approximatley weekly time steps.

In [29]:
# start by creating a new tuple for the first stress period
# the tuple needs to be in a sequence of stress period length, number of steps and mutiplier.
# you can change it directly or rebuild the object
tdis.perioddata.array[0] = (365.25,52,1)
tdis
sim.write_simulation()
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis"))]
#tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00")

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
# File generated by Flopy version 3.4.2 on 09/17/2023 at 10:05:09.
BEGIN options
  TIME_UNITS  days
  START_DATE_TIME  2023-12-31t00:00:00
END options

BEGIN dimensions
  NPER  1
END dimensions

BEGIN perioddata
     365.25000000  52       1.00000000
END perioddata



In [30]:
# alternativley just rebuild it
del tdis
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00",perioddata=[(365.25,52,1)])
tdis

package_name = MySim.tdis
filename = MySim.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = MySim

Block options
--------------------
time_units
{internal}
('days')

start_date_time
{internal}
('2023-12-31t00:00:00')


Block dimensions
--------------------
nper
{internal}
(1)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([(365.25, 52, 1.)],
          dtype=[('perlen', '<f8'), ('nstp', '<i4'), ('tsmult', '<f8')]))



Okay now we will try something a bit tricky with time discretization. We are going to build our stress period data using actual dates. We want an initial steady-state period followed by 20 years of monthly stress periods with monthly time steps followed by a single stress period of 100 years with approximatley monthly time steps. Our model will start on the 31/12/2023 with the steady-state period of 1 day. This means we want our transient period to start on the 01/01/2024

In [31]:
# we start with the 20 years of monthly stress periods, note the starting date used in the range.
dates = pd.date_range('2024-01-01','2043-01-01', freq='MS').tolist() # use pandas to create a list of timestamps in monthly frquency
dates

[Timestamp('2024-01-01 00:00:00'),
 Timestamp('2024-02-01 00:00:00'),
 Timestamp('2024-03-01 00:00:00'),
 Timestamp('2024-04-01 00:00:00'),
 Timestamp('2024-05-01 00:00:00'),
 Timestamp('2024-06-01 00:00:00'),
 Timestamp('2024-07-01 00:00:00'),
 Timestamp('2024-08-01 00:00:00'),
 Timestamp('2024-09-01 00:00:00'),
 Timestamp('2024-10-01 00:00:00'),
 Timestamp('2024-11-01 00:00:00'),
 Timestamp('2024-12-01 00:00:00'),
 Timestamp('2025-01-01 00:00:00'),
 Timestamp('2025-02-01 00:00:00'),
 Timestamp('2025-03-01 00:00:00'),
 Timestamp('2025-04-01 00:00:00'),
 Timestamp('2025-05-01 00:00:00'),
 Timestamp('2025-06-01 00:00:00'),
 Timestamp('2025-07-01 00:00:00'),
 Timestamp('2025-08-01 00:00:00'),
 Timestamp('2025-09-01 00:00:00'),
 Timestamp('2025-10-01 00:00:00'),
 Timestamp('2025-11-01 00:00:00'),
 Timestamp('2025-12-01 00:00:00'),
 Timestamp('2026-01-01 00:00:00'),
 Timestamp('2026-02-01 00:00:00'),
 Timestamp('2026-03-01 00:00:00'),
 Timestamp('2026-04-01 00:00:00'),
 Timestamp('2026-05-

In [32]:
perlens = [(dates[x]-dates[x-1]).days for x in range(1,len(dates))] # gives me the monthly period lengths in days. note use of .days
perlens # this will help to check if we have the correct starting values.
# Question: why do I start my range at 1 and not zero? Why not for x in dates
# Recall that we need to provide a list of tuples to the tdis object where the first item in the tuple is the stress period length.

[31,
 29,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 29,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 29,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 29,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 28,
 31,
 30,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 31,
 29,
 31,
 30,
 31,
 30,
 31,
 31,


In [33]:
# check the number of entries in period lengths and assgn it to a variable because we will need it later
numper = len(perlens) # checks the length of the list. It should be 228 because we have 20 * 12 stress periods.

In [34]:
stp = 1 # we'll see how the model stability goes with 1 time step per period to start off with 
# build the transient stress period data tuples for the tdis [ackage
_ = [(x,stp,1) for x in perlens] # note the final 1 in the tuple is the multiplier. We won't use it to begin with.

In [35]:
# add in the steady state stress period at the start as (1,1,1) and 100 year period at the end
# note my 100 year period has 1200 steps so sticking with approximatley monthly time steps
pdata = [(1,1,1), *_, (36525, 1200, 1)] # note the use of unpacking the _ tuple list
# now we have a single recovery period that is 100 years in length with approximatley mothly time steps
numper = len(pdata) # we are updating the value of numper here to include our steady-state and 100 year periods
print(numper) # print the value for viewing
pdata # view all the period data. Note that this is what we will provide to the 

230


[(1, 1, 1),
 (31, 1, 1),
 (29, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (28, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (28, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (28, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (29, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (28, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (30, 1, 1),
 (31, 1, 1),
 (31, 1, 1),
 (28, 1, 1),
 (31, 1, 1),
 (30, 1, 1),


As a reference we are going to create a dataframe for use later. This will provide a check for any transient behaviour for boundary conditions. For example if we know pumping is scheduled to commence in February 2025 then we might need to know which stress period this is. I say might here because there are alternative ways to control transient boundary conditions in MF6 that were not available in previous versions of MODFLOW.

In [36]:
ss = pd.Timestamp("2023-12-31") # set the date for the SS period
dates = [ss,*dates] # add the date
# Hmm, could we have done this originally and included the steady-state stress period when we got the tuple lengths? Try it yourself.
len(dates)

230

In [37]:
# Now lets build the dataframe
df = pd.DataFrame() # start with en empty dataframe
df['Date'] = dates # create a column with dates in it
df['SP'] = range(1,len(dates)+1) # create a column with the actual MF6 stress period number in it
df['Flopy_SP'] = range(len(dates)) # create a column with the Flopy zero-based stress period numbers in it
df.to_csv('model_timing.csv',index=None) # export the csv to the project folder
# if you want to keep this dataframe available for use later in the script then make a copy of it using a descriptive name
modtime_df = df.copy()

In [38]:
# Now we create the new tdis object
tdis = flopy.mf6.ModflowTdis(sim,time_units='days',start_date_time="2023-12-31T00:00:00",
                             perioddata=pdata,nper=numper)
tdis

package_name = MySim.tdis
filename = MySim.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = MySim

Block options
--------------------
time_units
{internal}
('days')

start_date_time
{internal}
('2023-12-31t00:00:00')


Block dimensions
--------------------
nper
{internal}
(230)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([(1.0000e+00,    1, 1.), (3.1000e+01,    1, 1.),
           (2.9000e+01,    1, 1.), (3.1000e+01,    1, 1.),
           (3.0000e+01,    1, 1.), (3.1000e+01,    1, 1.),
           (3.0000e+01,    1, 1.), (3.1000e+01,    1, 1.),
           (3.1000e+01,    1, 1.), (3.0000e+01,    1, 1.),
           (3.1000e+01,    1, 1.), (3.0000e+01,    1, 1.),
           (3.1000e+01,    1, 1.), (3.1000e+01,    1, 1.),
           (2.8000e+01,    1, 1.), (3.1000e+01,    1, 1.),
           (3.0000e+01,    1, 1.), (3.1000e+01,    1, 1.),
           (3.0000e+01,    1, 1.), (3.1000e+01,    1, 1.),
           (3.1000e+01,    1, 1.)

In [39]:
# lets write our simulation
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


In [40]:
# lets look at the tdis file
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis"))]

# File generated by Flopy version 3.4.2 on 09/17/2023 at 10:05:14.
BEGIN options
  TIME_UNITS  days
  START_DATE_TIME  2023-12-31t00:00:00
END options

BEGIN dimensions
  NPER  230
END dimensions

BEGIN perioddata
       1.00000000  1       1.00000000
      31.00000000  1       1.00000000
      29.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      31.00000000  1       1.00000000
      28.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      31.00000000  1      

In [41]:
from helpers import ws1_mod_trans 
# now we are importing the same model as before but with one extra package being built. 
# What is the new package? Why is it needed now?

In [42]:
# now lets run the transient version of the same model 
ws1_mod_trans(sim,tdis) # remember this is only putting the objects in memory you havent created the files yet

building ims package
building gwf package
building dis package
building ic package
building npf package
building sto package
building chd package
building oc package


In [43]:
sim.write_simulation() # write the simulation files
sim.run_simulation() # run the simulation

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model MyModel...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package sto...
    writing package chd_0...
INFORMATION: maxbound in ('gwf6', 'chd', 'dimensions') changed to 2 based on size of stress_period_data
    writing package oc...
FloPy is using the following executable to run the model: ..\..\..\..\mffiles\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.4.2 06/28/2023

   MODFLOW 6 compiled Jul 05 2023 20:31:54 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Althoug

(True, [])

Ideally you should be building models that are very stable but in some instances you may have to use an unstable model or you have advanced stress packages that are causing you some strife so you may opt to use adaptive time stepping. This will automatically change the time step length in an attempt to solve the model for you. Note this is one strategy to get to a solution for your model there are other options too. You may also choose to set this up immediatley from the get go but there are some caveats to doing this with regards to model output that you should be aware of. Take some time to read the ATS section in the mf6io.pdf to make sure that you understand how it all works. I'll provide some brief descriptions below.

In [44]:
# We need to provide the tdis object with some data for the ATS package in order for it to be created for us.
# in this example we are only going to want ATS active during the monthly stress periods
# first we define which stress perios we want the ATS active for
iperats = [x for x in range(1,numper-1)] # what am i doing here exactly?
dt0 = 0 # Now we say what we want the initial time step for the stress period to be. An entry of zero is special. Read up why.
dtmin = 0.01 # next we need to define the minimum time step length we will allow. Recall our time units are days.
# This next one requires the most thought and depending on how much your model is struggling to converge may require adjusting.
# We will use a recommended value for now
dtadj = 3.0
# The last variable is also worth reading up about, it controls how the time step length is reduced when your model fails to converge
# for brevity we will use the recommended value
dtfailadj = 5.0
# now we create our input data list of tuples. Check the mf6io.pdf for the entries to make sure we have them ordered correctly.
ats_pdata = [[x,dt0,dtmin,y,dtadj,dtfailadj] for x,y in zip(iperats,perlens)] # What are x and y here?
# now we need to pass this on to the tdis package and re-write

In [45]:
# now we initialize the ats in the existing tdis object
tdis.ats.initialize(maxats=len(ats_pdata),perioddata=ats_pdata)

In [46]:
sim.write_simulation() # look for the ats 

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing package MySim.tdis.ats...
  writing model MyModel...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package sto...
    writing package chd_0...
    writing package oc...


In [47]:
# lets look at our tdis input file now
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis"))]

# File generated by Flopy version 3.4.2 on 09/17/2023 at 10:05:19.
BEGIN options
  TIME_UNITS  days
  START_DATE_TIME  2023-12-31t00:00:00
  ATS6  FILEIN  MySim.tdis.ats
END options

BEGIN dimensions
  NPER  230
END dimensions

BEGIN perioddata
       1.00000000  1       1.00000000
      31.00000000  1       1.00000000
      29.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      31.00000000  1       1.00000000
      28.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.00000000
      30.00000000  1       1.00000000
      31.00000000  1       1.0000

In [48]:
# lets look at our ATS input file now
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"MySim.tdis.ats"))]

# File generated by Flopy version 3.4.2 on 09/17/2023 at 10:05:19.
BEGIN dimensions
  MAXATS  228
END dimensions

BEGIN perioddata
  2       0.00000000       0.01000000      31.00000000       3.00000000       5.00000000
  3       0.00000000       0.01000000      29.00000000       3.00000000       5.00000000
  4       0.00000000       0.01000000      31.00000000       3.00000000       5.00000000
  5       0.00000000       0.01000000      30.00000000       3.00000000       5.00000000
  6       0.00000000       0.01000000      31.00000000       3.00000000       5.00000000
  7       0.00000000       0.01000000      30.00000000       3.00000000       5.00000000
  8       0.00000000       0.01000000      31.00000000       3.00000000       5.00000000
  9       0.00000000       0.01000000      31.00000000       3.00000000       5.00000000
  10       0.00000000       0.01000000      30.00000000       3.00000000       5.00000000
  11       0.00000000       0.01000000      31.00000000       3.000

In [49]:
# Lest run it
sim.run_simulation()

FloPy is using the following executable to run the model: ..\..\..\..\mffiles\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.4.2 06/28/2023

   MODFLOW 6 compiled Jul 05 2023 20:31:54 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Govern

(True, [])

In [50]:
# lets clear everything again
for x in [sim,tdis]: # loop through a list of objects we created that we want to remove
    del x # use the del statement to remove the objects, this clears them from memory
flist = [x for x in os.listdir(model_f)] # gets a list of all the files in the model folder
_ = [os.remove(os.path.join(model_f,f)) for f in flist] # removes all the files in the model folder, faster as list comprehension? Maybe.
# now check that it worked
flist = [x for x in os.listdir(model_f)] # get the files again
flist # should see an empty list

[]